# Library and Data Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd                                            
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 

import sklearn.model_selection as ms     
from sklearn import tree                                   
from sklearn.metrics import classification_report, confusion_matrix 
from IPython.display import Image                 
import pydotplus                                            
import os  
import statsmodels.api as ms
import matplotlib.pyplot as plt
import seaborn as sns

# import data
file_path = '/content/drive/MyDrive/INFO-614 Group Project/google_playstore.xlsx'
df = pd.read_excel(file_path)
df.head()


# Removing Missing Values

In [ ]:
display(df.isna().sum())
df = df.dropna()

In [ ]:
df.shape

# Sampling 25% of Data Set

In [ ]:
import random
random.seed(1)

df = df.sample(n=222174, random_state=1)

df.shape

# Review Correlation Matrix

In [ ]:
# Correlation Matrix is used here to figure out if if any multicollinearity exist
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(), annot=True,cmap='Reds', fmt='.2f')

# Subseting the Dataframe

In [ ]:
df = df[['category','rating_count', 'installs', 'price', 'size', 'content_rating', 'ad', 'rating']]
# We may enhance the accuracy by reducing the dimensionality but most of the features are significantly affecting the labelled data so we prefer to keep them in training model.

# Data PreProcessing

In [ ]:
df['category'] = df['category'].astype('category').cat.codes
df['size'] = df['size'].astype('category').cat.codes
df['content_rating'] = df['content_rating'].astype('category').cat.codes

df.loc[df['rating'] <= 4.5, 'rating'] = 0 
df.loc[df['rating'] > 4.5, 'rating'] = 1

display(df['rating'].value_counts())

In [ ]:
import seaborn as sns
sns.histplot(df['rating']).set_title('rating Distribution')

# Data Train and Test Set Split

In [ ]:
import sklearn.model_selection as ms 
X = (df.iloc[:,:-1])
y = (df.iloc[:, -1])
y = y.astype('int')


X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size = 0.3, random_state = 1)

# Tree Induction

In [ ]:
dt_clf = DecisionTreeClassifier( criterion='gini', max_depth=6, min_samples_split = 10, random_state=1)

dt_clf = dt_clf.fit(X_train, y_train)
                    
y_pred = dt_clf.predict(X_test)

# Tree Visualization

In [ ]:
import io
import pydot
from IPython.core.display import Image
from sklearn.tree import export_graphviz
import matplotlib as mpl



def draw_decision_tree(model):
    dot_buf = io.StringIO()
    export_graphviz(model, out_file=dot_buf, feature_names=feature_names)
    graph = pydot.graph_from_dot_data(dot_buf.getvalue())[0]
    image = graph.create_png()
    return Image(image)


def plot_decision_regions(X, y, model, title):
    resolution = 0.01
    markers = ('s', '^', 'o')
    colors = ('red', 'blue', 'lightgreen')
    cmap = mpl.colors.ListedColormap(colors)

    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = model.predict(
        np.array([xx1.ravel(), xx2.ravel()]).T).reshape(xx1.shape)

    plt.contour(xx1, xx2, Z, cmap=mpl.colors.ListedColormap(['k']))
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1], alpha=0.8,
                    c=[cmap(idx)], marker=markers[idx], s=80, label=cl)

    plt.xlabel(data.feature_names[2])
    plt.ylabel(data.feature_names[3])
    plt.legend(loc='upper left')
    plt.title(title)

    return Z

In [ ]:
feature_names = df.columns.tolist()
feature_names = feature_names[0:7]
target_name = np.array(['Low', 'High'])

dt_dot_data = tree.export_graphviz(dt_clf, out_file = None,
                                  feature_names = feature_names,
                                  class_names = target_name,
                                  filled = True, rounded = True,
                                  special_characters = True)

dt_graph = pydotplus.graph_from_dot_data
draw_decision_tree(dt_clf)

# Validation

In [ ]:
import sklearn.metrics as mt

print('Train_Accuracy: ', dt_clf.score(X_train, y_train),'\n')

accuracy = mt.accuracy_score(y_test, y_pred)
recall = mt.recall_score(y_test, y_pred)
precision = mt.precision_score(y_test, y_pred)
f1_score = mt.f1_score(y_test, y_pred)
matrix = mt.confusion_matrix(y_test, y_pred)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

# Cross-Validation

In [ ]:



from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import cross_val_predict


y_pred_cross = cross_val_predict(dt_clf, X, y, cv=10)
accuracy = mt.accuracy_score(y, y_pred_cross)
recall = mt.recall_score(y, y_pred_cross)
precision = mt.precision_score(y, y_pred_cross)
f1_score = mt.f1_score(y, y_pred_cross)
matrix = mt.confusion_matrix(y, y_pred_cross)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

# Parameter Tunning (Pruning)

In [ ]:
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


mean_test_accuracy = []
train_accuracy = []
for max_depth in np.arange(3, 8):
    model1 = DecisionTreeClassifier(max_depth=max_depth).fit(X, y)
    train_accuracy.append(accuracy_score(y, model1.predict(X)))
    mean_test_accuracy.append(cross_val_score(model1, X, y, scoring="accuracy", cv=10).mean())
    
plt.plot(np.arange(3, 8), train_accuracy)
plt.plot(np.arange(3, 8), mean_test_accuracy)
plt.legend(["train_accuracy", "test_accuracy"], loc ="lower right")


In [ ]:
from sklearn.model_selection import GridSearchCV

t_clf = DecisionTreeClassifier(random_state=0)
parameters = {'max_depth': [4, 8, 10, 12],
              'min_samples_split': [5, 10, 15],
              'splitter': ['best', 'random'],}

grid_dt = GridSearchCV(dt_clf, param_grid = parameters, cv = 10 )

grid_dt.fit(X_train, y_train)

result = pd.DataFrame(grid_dt.cv_results_['params'])
result['mean_test_score'] = grid_dt.cv_results_['mean_test_score']
result.sort_values(by='mean_test_score', ascending=False)

# Tree Optimization 

In [ ]:
# Model 2
dt_clf = DecisionTreeClassifier(max_depth=8, min_samples_split = 10, splitter= 'best', random_state=1)

dt_clf = dt_clf.fit(X_train, y_train)
                    
y_pred = dt_clf.predict(X_test)

feature_names = df.columns.tolist()
feature_names = feature_names [0:8]
target_name = np.array (['1', '0'])

In [ ]:
# Validation
import sklearn.metrics as mt

print('Train_Accuracy: ', dt_clf.score(X_train, y_train),'\n')

accuracy = mt.accuracy_score(y_test, y_pred)
recall = mt.recall_score(y_test, y_pred)
precision = mt.precision_score(y_test, y_pred)
f1_score = mt.f1_score(y_test, y_pred)
matrix = mt.confusion_matrix(y_test, y_pred)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

In [ ]:
# Cross-Validation 
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import cross_val_predict

y_pred_cross = cross_val_predict(dt_clf, X, y, cv=10)
accuracy = mt.accuracy_score(y, y_pred_cross)
recall = mt.recall_score(y, y_pred_cross)
precision = mt.precision_score(y, y_pred_cross)
f1_score = mt.f1_score(y, y_pred_cross)
matrix = mt.confusion_matrix(y, y_pred_cross)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

In [ ]:
# Model 3
dt_clf = DecisionTreeClassifier(max_depth = 8, min_samples_split = 15, splitter= 'best', random_state=0)

dt_clf = dt_clf.fit(X_train, y_train)
                    
y_pred = dt_clf.predict(X_test)

feature_names = df.columns.tolist()
feature_names = feature_names [0:8]
target_name = np.array (['1', '0'])

In [ ]:
# Validation
import sklearn.metrics as mt

print('Train_Accuracy: ', dt_clf.score(X_train, y_train),'\n')

accuracy = mt.accuracy_score(y_test, y_pred)
recall = mt.recall_score(y_test, y_pred)
precision = mt.precision_score(y_test, y_pred)
f1_score = mt.f1_score(y_test, y_pred)
matrix = mt.confusion_matrix(y_test, y_pred)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

In [ ]:
# Cross-Validation
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import cross_val_predict

y_pred_cross = cross_val_predict(dt_clf, X, y, cv=10)
accuracy = mt.accuracy_score(y, y_pred_cross)
recall = mt.recall_score(y, y_pred_cross)
precision = mt.precision_score(y, y_pred_cross)
f1_score = mt.f1_score(y, y_pred_cross)
matrix = mt.confusion_matrix(y, y_pred_cross)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

In [ ]:
# Model 4
dt_clf = DecisionTreeClassifier(max_depth=5, min_samples_split = 15, splitter= 'random', random_state=0)

dt_clf = dt_clf.fit(X_train, y_train)
                    
y_pred = dt_clf.predict(X_test)

feature_names = df.columns.tolist()
feature_names = feature_names [0:8]
target_name = np.array (['1', '0'])

In [ ]:
# Validation
import sklearn.metrics as mt

print('Train_Accuracy: ', dt_clf.score(X_train, y_train),'\n')

accuracy = mt.accuracy_score(y_test, y_pred)
recall = mt.recall_score(y_test, y_pred)
precision = mt.precision_score(y_test, y_pred)
f1_score = mt.f1_score(y_test, y_pred)
matrix = mt.confusion_matrix(y_test, y_pred)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

In [ ]:
# Cross-Validation
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import cross_val_predict

y_pred_cross = cross_val_predict(dt_clf, X, y, cv=10)
accuracy = mt.accuracy_score(y, y_pred_cross)
recall = mt.recall_score(y, y_pred_cross)
precision = mt.precision_score(y, y_pred_cross)
f1_score = mt.f1_score(y, y_pred_cross)
matrix = mt.confusion_matrix(y, y_pred_cross)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

# Logistic Regression Model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn.model_selection as ms
from sklearn.metrics import confusion_matrix
import statsmodels.api as ms



In [ ]:
# Normalization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = (df.iloc[:,:-1])
y = (df.iloc[:, -1])
y = y.astype('int')

X = scaler.fit_transform(X)

# Train and Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

# LR Model Fit

In [ ]:
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as mt

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Train_Accuracy: ', model.score(X_train, y_train),'\n')

# LR Table

In [ ]:
model = ms.Logit(y_train, X_train)
results = model.fit(method = "newton")
features = list(df.iloc[:, 0:-1].columns) 
results.summary(xname=features)

# LR Test

In [ ]:
accuracy = mt.accuracy_score(y_test, y_pred)
recall = mt.recall_score(y_test, y_pred)
precision = mt.precision_score(y_test, y_pred)
f1_score = mt.f1_score(y_test, y_pred)
matrix = mt.confusion_matrix(y_test, y_pred)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

# LR Cross-Validation

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import cross_val_predict

model = LogisticRegression()

y_pred_cross = cross_val_predict(model, X, y, cv=10)
accuracy = mt.accuracy_score(y, y_pred_cross)
recall = mt.recall_score(y, y_pred_cross)
precision = mt.precision_score(y, y_pred_cross)
f1_score = mt.f1_score(y, y_pred_cross)
matrix = mt.confusion_matrix(y, y_pred_cross)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

In [ ]:
# We tried optimizing LR by Removing the features with p-value > 0.05 but no significant changes were observed, so we opt out this model optimization
#df = df[['category','size', 'content_rating', 'ad', 'rating']]

# KNN Model

In [ ]:
import pandas as pd                                            
import numpy as np
from sklearn.model_selection import train_test_split 
import sklearn.model_selection as ms             
from sklearn.metrics import confusion_matrix 
import statsmodels.api as sm   

# Normalization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = (df.iloc[:,:-1])
y = (df.iloc[:, -1])
y = y.astype('int')

X = scaler.fit_transform(X)

# Set Train and Test Set
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size = 0.3, random_state = 1)

In [ ]:
# KNN Model 1 Fit
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors = 7)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print('Train_Accuracy: ', model.score(X_train, y_train),'\n')

In [ ]:
# Test
import sklearn.metrics as mt

accuracy = mt.accuracy_score(y_test, y_pred)
recall = mt.recall_score(y_test, y_pred)
precision = mt.precision_score(y_test, y_pred)
f1_score = mt.f1_score(y_test, y_pred)
matrix = mt.confusion_matrix(y_test, y_pred)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

In [ ]:
# Cross-Validation
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import cross_val_predict

model = KNeighborsClassifier(n_neighbors = 7)

y_pred_cross = cross_val_predict(model, X, y, cv=10)
accuracy = mt.accuracy_score(y, y_pred_cross)
recall = mt.recall_score(y, y_pred_cross)
precision = mt.precision_score(y, y_pred_cross)
f1_score = mt.f1_score(y, y_pred_cross)
matrix = mt.confusion_matrix(y, y_pred_cross)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

# KNN Parameter Tuning

In [ ]:
import matplotlib.pyplot as plt

k_range = range(1,23)

accuracy_list = []

for k in k_range:
    model = KNeighborsClassifier(n_neighbors = k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy_list.append(mt.accuracy_score(y_test, y_pred))

plt.plot(k_range, accuracy_list, 'o--', color = 'blue')
plt.xlabel("k")
plt.ylabel("test accuracy")
plt.show()

# KNN Model Optimization

In [ ]:
# KNN Model 2
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors = 8)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print('Train_Accuracy: ', model.score(X_train, y_train),'\n')

In [ ]:
# Test
import sklearn.metrics as mt

accuracy = mt.accuracy_score(y_test, y_pred)
recall = mt.recall_score(y_test, y_pred)
precision = mt.precision_score(y_test, y_pred)
f1_score = mt.f1_score(y_test, y_pred)
matrix = mt.confusion_matrix(y_test, y_pred)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

In [ ]:
# Cross-Validation
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import cross_val_predict

model = KNeighborsClassifier(n_neighbors = 8)

y_pred_cross = cross_val_predict(model, X, y, cv=10)
accuracy = mt.accuracy_score(y, y_pred_cross)
recall = mt.recall_score(y, y_pred_cross)
precision = mt.precision_score(y, y_pred_cross)
f1_score = mt.f1_score(y, y_pred_cross)
matrix = mt.confusion_matrix(y, y_pred_cross)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

In [ ]:
# KNN Model 3
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors = 11)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print('Train_Accuracy: ', model.score(X_train, y_train),'\n')

In [ ]:
# Test
import sklearn.metrics as mt

accuracy = mt.accuracy_score(y_test, y_pred)
recall = mt.recall_score(y_test, y_pred)
precision = mt.precision_score(y_test, y_pred)
f1_score = mt.f1_score(y_test, y_pred)
matrix = mt.confusion_matrix(y_test, y_pred)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

In [ ]:
# Cross-Validation
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import cross_val_predict

model = KNeighborsClassifier(n_neighbors = 11)

y_pred_cross = cross_val_predict(model, X, y, cv=10)
accuracy = mt.accuracy_score(y, y_pred_cross)
recall = mt.recall_score(y, y_pred_cross)
precision = mt.precision_score(y, y_pred_cross)
f1_score = mt.f1_score(y, y_pred_cross)
matrix = mt.confusion_matrix(y, y_pred_cross)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

In [ ]:
# KNN Model 4
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors = 19)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print('Train_Accuracy: ', model.score(X_train, y_train),'\n')

In [ ]:
# Test
import sklearn.metrics as mt

accuracy = mt.accuracy_score(y_test, y_pred)
recall = mt.recall_score(y_test, y_pred)
precision = mt.precision_score(y_test, y_pred)
f1_score = mt.f1_score(y_test, y_pred)
matrix = mt.confusion_matrix(y_test, y_pred)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

In [ ]:
# Cross-Validation
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import cross_val_predict

model = KNeighborsClassifier(n_neighbors = 19)

y_pred_cross = cross_val_predict(model, X, y, cv=10)
accuracy = mt.accuracy_score(y, y_pred_cross)
recall = mt.recall_score(y, y_pred_cross)
precision = mt.precision_score(y, y_pred_cross)
f1_score = mt.f1_score(y, y_pred_cross)
matrix = mt.confusion_matrix(y, y_pred_cross)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)